#### 简单层的实现

层的实现主要是实现forward()和backward()两个方法（接口）。
forward()对应正向传播，backward()对应反向传播。

乘法节点称为乘法层，加法节点称为加法层。


In [ ]:
#乘法层的实现
class MulLayer:
    #初始化实例变量x和y，保存正向传播时的输入值
    def __init__(self):
        self.x = None
        self.y = None
        
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        return out
    
    def backward(self, dout):
        dx = dout * self.y #翻转x,y
        dy = dout * self.x
        return dx, dy

In [ ]:
apple = 100
apple_num = 2
tax = 1.1

#layer
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

#forward
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price)

In [ ]:
#各个变量的导数可由backward()导出
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

In [ ]:
#加法层的实现
class AddLayer:
    def __init__(self):
        pass
    
    def forward(self, x, y):
        out = x + y
        return out
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

In [ ]:
applle = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

#layer
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

#forward
apple_price = mul_apple_layer.forward(applle, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

#backward
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(price)
print(dapple_num, dapple, dorange_num, dorange, dtax)

#### 激活函数层的实现

In [ ]:
#RelU
class RelU:
    def __init__(self):
        self.mask = None#mask是由TRUE\FALSE组成的数组
        
    def forward(self, x):
        self.mask = (x <= 0)#mask数组中只有x<=0的元素为TRUE，其余为FALSE
        out = x.copy()
        out[self.mask] = 0
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        return dx

In [ ]:
import numpy as np

x = np.array([[1.0, -0.5], [-2.0, 3.0]])
print(x)
mask = (x <= 0)
print(mask)

In [ ]:
#sigmoid
class Sigmoid:
    def __init__(self):
        self.out = None
        
    def forward(self, x):
        self.out = 1 / (1 + np.exp(-x))
        return self.out
    
    def backward(self, dout):
        dx = dout * self.out * (1 - self.out)
        return dx

#### Affine/Softmax层的实现

In [ ]:
class Affine:
    def __init__(self, W, b):
        self.W= W
        self.b= b
        self.x= None
        self.dW= None
        self.db= None
        
    def forward(self, x):
        self.x= x
        out= np.dot(self.x, self.W) + self.b
        return out
    
    def backward(self, dout):
        dx= np.dot(dout, self.W.T)
        self.dW= np.dot(self.x.T, dout)
        self.db= np.sum(dout, axis=0)
        return dx

In [ ]:
class SoftmaxwithLoss:
    def __init__(self):
        self.loss = None
        self.y = None#softmax输出
        self.t = None#标签（监督数据）
       
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss 
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        return dx

#### 误差反向传播法的实现

In [20]:
#TwoLayerNet的代码实现
import sys,os
sys.path.append(os.pardir)
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        #初始化权重
        self.params = {} #定义一个字典，字典定义了键和值之间一一对应的关系
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

        #生成层
        self.layers = OrderedDict() #有序字典 
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        # self.layers = OrderedDict([('Affine1', <common.layers.Affine object at 0x000002B247D69A60>),\
                                    # ('Relu1', <common.layers.Relu object at 0x000002B247B54970>), \
                                    # ('Affine2', <common.layers.Affine object at 0x000002B247EB1490>)])

        self.lastLayer = SoftmaxWithLoss()

    def predict(self, x):
        #dict.values()方法获取字典里的所有值
        for layer in self.layers.values():
                x = layer.forward(x)
        return x

    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)

    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy

    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        return grads

    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse() #翻转
        for layer in layers:
            dout = layer.backward(dout)#Affine层的backward计算了dW和db,且dW和db是实例变量

        # 设定        
        grads = {}                
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads

In [21]:
#梯度确认
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

#dict.keys()方法获取字典里的所有键
print(grad_numerical.keys())

#求各个权重的绝对误差的平均值
for key in grad_numerical.keys():
    diff = np.average(np.abs(grad_backprop[key] - grad_numerical[key]))
    print(key + ":" + str(diff))

dict_keys(['W1', 'b1', 'W2', 'b2'])
W1:4.190801144830887e-10
b1:2.658191441890553e-09
W2:5.622162615152322e-09
b2:1.397974159753068e-07


In [23]:
#神经网络学习的实现
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

#循环10000次，每次使用100个样本进行训练，总共遍历整个训练集10000*100/60000次
iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

#损失函数列表
train_loss_list = []
#准确率列表
train_acc_list = []
test_acc_list = []

#每一个epoch的迭代次数
iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    #通过误差反向传播法计算梯度
    grad = network.gradient(x_batch, t_batch)
    
    #更新参数
    for key in grad.keys():
        network.params[key] -= learning_rate * grad[key]
        
    #记录训练集的损失函数和准确率
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train acc, test acc | " + str(train_acc) + ", " + str(test_acc))

train acc, test acc | 0.07191666666666667, 0.067
train acc, test acc | 0.903, 0.9083
train acc, test acc | 0.9221666666666667, 0.9251
train acc, test acc | 0.9317333333333333, 0.9301
train acc, test acc | 0.9410833333333334, 0.9399
train acc, test acc | 0.9489833333333333, 0.9471
train acc, test acc | 0.9532333333333334, 0.9524
train acc, test acc | 0.9572, 0.9548
train acc, test acc | 0.9616166666666667, 0.9596
train acc, test acc | 0.96425, 0.9606
train acc, test acc | 0.96755, 0.9642
train acc, test acc | 0.9702166666666666, 0.9656
train acc, test acc | 0.97135, 0.9648
train acc, test acc | 0.9735333333333334, 0.9674
train acc, test acc | 0.9738833333333333, 0.9664
train acc, test acc | 0.97505, 0.9675
train acc, test acc | 0.9775, 0.9692
